In [1]:
%pip install gym[atari]
%pip install ale_py
%pip install autorom[accept-rom-license]
%pip install torch
%pip install gym[box2d]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

  Using cached pygame-2.1.0-cp39-cp39-win_amd64.whl (4.8 MB)
  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for box2d-py
Failed to build box2d-py
  Running setup.py install for box2d-py: started
  Running setup.py install for box2d-py: finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      Using setuptools (version 63.4.1).
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-39
      creating build\lib.win-amd64-cpython-39\Box2D
      copying library\Box2D\Box2D.py -> build\lib.win-amd64-cpython-39\Box2D
      copying library\Box2D\__init__.py -> build\lib.win-amd64-cpython-39\Box2D
      creating build\lib.win-amd64-cpython-39\Box2D\b2
      copying library\Box2D\b2\__init__.py -> build\lib.win-amd64-cpython-39\Box2D\b2
      running build_ext
      building 'Box2D._Box2D' extension
      swigging Box2D\Box2D.i to Box2D\Box2D_wrap.cpp
      swig.exe -python -c++ -IBox2D -small -O -includeall -ignoremissing -w201 -globals b2Globals -outdir library\Box2D -keyword -w511 -D_SWIG_KWARGS -o Box2D\Box2D_wrap.cpp Box2D\Box2D.i
      Box2D\Common\b2Math

In [2]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import gym
import random as rd

In [3]:
class LinearReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.pre_pos = 6
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)
        self.zieldurchlaeufe = 0
        self.crashCount=0

    def reward(self, reward):
        ram = env.unwrapped.ale.getRAM()
        current_pos = ram[14]
        #print(current_pos)
        #print(ram[106])
        reward=0
        reward-=500/current_pos
        reward+=current_pos
        if(ram[106]>110):
            reward+=10000
        self.pre_pos = current_pos
        if(ram[106]>0 and self.pre_pos<170):
            #print("Crash!: ",ram)
            #reward-=5
            self.crashCount+=1
        return np.clip(reward, self.min_reward, self.max_reward)

    def getZieldurchlaeufe(self):
        return self.zieldurchlaeufe
    def getCrashCount(self):
        return self.crashCount
    def targetReset(self):
        self.crashCount= 0
        self.zieldurchlaeufe = 0
    

In [4]:
class DeepQNetwork(nn.Module):
    def __init__(self, lr, input_dims, fc1_dims, fc2_dims, n_actions):
        #Vererbung: Aufruf des Konstruktors der Superklasse
        super(DeepQNetwork, self).__init__()
        #Speichern der übergebenen Parameter
        self.input_dims = input_dims #Dimension des Inputs
        self.fc1_dims = fc1_dims #Dimension des ersten fully connected Layer
        self.fc2_dims = fc2_dims #Dimension des zweiten fully connected Layer
        self.n_actions = n_actions #Anzahl der möglichen Aktionen
        self.af=nn.LeakyReLU(0.1)#Leaky ReLU Activation Function (bc negative Reward)

        #Lineare Transformation
        #Pro Layer wird eine lineare Transformation angewandt
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims) #* entpackt die input_dims
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.n_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=lr)

        #Mean squared error loss
        self.loss = nn.MSELoss()

        #Choosing GPU if possible
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')

        #TODO:
        self.to(self.device)

    def forward(self, state):
        # #Activation function (RELU)
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        # x = F.leaky_relu(self.fc1(state))
        # x = F.leaky_relu(self.fc2(x))
        #Only applied to the first two layers, not the output
        actions = self.fc3(x)
        return actions

In [5]:
class Agent():
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size, n_actions, max_mem_size = 1000000, eps_end = 0.01, eps_dec = 1.5e-4):
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.lr = lr
        self.action_space = [i for i in range(n_actions)]
        self.mem_size = max_mem_size
        self.batch_size = batch_size
        self.mem_counter = 0
        
        self.Q_eval = DeepQNetwork(self.lr, n_actions = n_actions, input_dims = input_dims, fc1_dims = 256, fc2_dims= 256)
        self.state_memory = np.zeros((self.mem_size, *input_dims), dtype = np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims), dtype = np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype = np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype = np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype = np.bool_)
    
    def store_transition(self, state, action, reward, state_, done):
        #Memory wird wieder von vorne überschrieben, wenn voll
        index = self.mem_counter % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = done

        self.mem_counter += 1
    
    def choose_action(self, observation):
        if np.random.random() > self.epsilon:
            #Exploitation instead of exploration
            state = T.tensor(np.array([observation])).to(self.Q_eval.device)
            actions = self.Q_eval.forward(state)
            #Choosing the best actionu
            action = T.argmax(actions).item()
        else:
            #Exploration instead of exploitation
            action = np.random.choice(self.action_space,p=[.1,.8,.1])#Stehen, vor, zurueck

        return action

    def learn(self):
        if self.mem_counter < self.batch_size:
            #Zu Beginn ist der Speicher noch leer
            #deshalb wird der Speicher gefüllt, bevor der Agent anfängt zu lernen
            return
        
        #Gradienten werden auf null gesetzt.
        #Andernfalls werden die Gradienten in Pytorch kummuliert
        self.Q_eval.optimizer.zero_grad()

        max_mem = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_mem, self.batch_size, replace = False)

        batch_index = np.arange(self.batch_size, dtype = np.int32)

        state_batch = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)
        action_batch = self.action_memory[batch]

        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        q_next = self.Q_eval.forward(new_state_batch)
        q_next[terminal_batch] = 0.0
        q_target = reward_batch + self.gamma * T.max(q_next, dim = 1)[0]

        loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()

        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min

    def getModel(self):
        return self.Q_eval

    def setModel(self, path):
        self.Q_eval = T.load(path)
    def resetEps(self):
        self.epsilon=1.0

In [6]:
import gym
import numpy as np
from matplotlib import pyplot as plt

name="TestRAMReLuv1_LinearReward0_"

#env = ClipReward(gym.make("ALE/Freeway-v5", difficulty = 1, mode = 3, obs_type = "ram",frameskip=1,repeat_action_probability=0), -1000, 10000)#warum nicht in der n_games Schleife?
agent = Agent(gamma = 0.99, epsilon = 1.0, batch_size = 64, n_actions = 3, eps_end = 0.01, input_dims = [128], lr = 0.01)
scores, eps_history, zieldurchlaeufe, crashes = [], [], [], []
n_games = 200
env = LinearReward(gym.make("ALE/Freeway-v5", difficulty = 1, mode = 3, obs_type = "ram"), -1000, 10000)
for i in range(n_games):
    
    score = 0
    done = False
    observation = env.reset()[0]
    while not done:
        action = agent.choose_action(np.float32(observation))
        observation_=observation
        #Info2 eingefügt
        obs, reward, done, info, info2 = env.step(action)
        score += reward
        agent.store_transition(observation_, action, reward, observation, done)
        agent.learn()
        observation = observation_
    scores.append(score)
    eps_history.append(agent.epsilon)
    zieldurchlaeufe.append(LinearReward.getZieldurchlaeufe(env))
    crashes.append(LinearReward.getCrashCount(env))
    print(LinearReward.getZieldurchlaeufe(env))
    print('episode ', i, 'score %.2f' % score, 'epsilon %.2f' % agent.epsilon,"crash count: %i" %LinearReward.getCrashCount(env))
    if(i%50==0): # Save model every 50 games in case learning needs to be interrupted
        agent.resetEps()
        s=f"C:\\Users\Simon\projektSeminar\DeepLearningProjekSeminar-main V1\{name}{i}.pth"
        T.save(Agent.getModel(agent), s)
x = [i + 1 for i in range(n_games)]


T.save(Agent.getModel(agent), f"C:\\Users\Simon\projektSeminar\Modelle\{name}.pth")

plt.plot(x, scores)
plt.ylabel="Scores"
plt.show()
plt.plot(x,zieldurchlaeufe)
plt.ylabel="Zieldurchlaeufe"
plt.show()
plt.plot(x,eps_history)
plt.ylabel="Epsilon"
plt.show()
plt.plot(x,crashes)
plt.ylabel="Crashes"
plt.show()

0
episode  0 score -99216.03 epsilon 0.70 crash count: 1046
0
episode  1 score -107170.93 epsilon 0.69 crash count: 2189
